In [ ]:
%%capture
!pip3 install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu116
!pip3 install rdkit
!pip3 install git+https://github.com/molecularsets/moses.git

In [ ]:
import pandas as pd
import numpy as np
import torch
torch.cuda.set_device(0)
import sys
import os
import tarfile
import gzip
import shutil
from rdkit import Chem
from rdkit.Chem import AllChem
import torch.nn as nn
import torch.optim as optim
from rdkit.Chem import Draw, inchi
from rdkit.Chem.Draw import DrawingOptions
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, QED, RDConfig
from rdkit.DataStructs import TanimotoSimilarity
import os
import sys
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer
from moses.utils import disable_rdkit_log
from moses.metrics.utils import get_mol
disable_rdkit_log()

/usr/local/lib/python3.10/dist-packages/moses/metrics/utils.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _mcf.append(_pains, sort=True)['smarts'].values]


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
os.chdir('/content/gdrive/My Drive/GENTRL-private')

In [ ]:
# Activity prediction model definition
class EnhancedCNN(nn.Module):
    def __init__(self):
        super(EnhancedCNN, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(256, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(512, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(2048, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(4096, 1)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x


In [ ]:
model = EnhancedCNN()
model.load_state_dict(torch.load('./models/esr1_active-pred_model.pth'))
model = model.to(device)
model.eval()

EnhancedCNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [ ]:
# TO USE FOR PREDICTION - COMPLETE METRICS

import torch
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import numpy as np
from collections import Counter

def generate_chemception_image(mol, dim=80):
    # Initialize the image to be of size dim x dim with all zeros (vacuum)
    image = np.zeros((dim, dim, 3))
    mol = Chem.AddHs(mol)

    # Get molecule coordinates after embedding
    if AllChem.EmbedMolecule(mol, Chem.AllChem.ETKDG()) == -1:
        # Embedding failed
        print("Embedding failed for molecule with SMILES:", Chem.MolToSmiles(mol))
        return None

    AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    conf = mol.GetConformer(0)

    # Scale and translate coordinates to fit the image dimensions
    xs = [conf.GetAtomPosition(i).x for i in range(mol.GetNumAtoms())]
    ys = [conf.GetAtomPosition(i).y for i in range(mol.GetNumAtoms())]
    x_min, x_max = min(xs), max(xs)
    y_min, y_max = min(ys), max(ys)
    x_range, y_range = x_max - x_min, y_max - y_min

    scale = 0.95 * dim / max(x_range, y_range)  # Scale down by 95% to provide some buffer
    x_trans, y_trans = (x_max + x_min)/2, (y_max + y_min)/2

    # Assign values to the grid based on atomic properties
    for i in range(mol.GetNumAtoms()):
        x, y = conf.GetAtomPosition(i).x, conf.GetAtomPosition(i).y
        x, y = int((x - x_trans) * scale + dim/2), int((y - y_trans) * scale + dim/2)

        # Make sure coordinates are inside the image
        x = max(0, min(dim-1, x))
        y = max(0, min(dim-1, y))

        # Atomic mass channel (channel 0)
        atom = mol.GetAtomWithIdx(i)
        image[y, x, 0] = atom.GetMass()

    # Assign values based on bonds
    for bond in mol.GetBonds():
        start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        x_start, y_start = conf.GetAtomPosition(start).x, conf.GetAtomPosition(start).y
        x_end, y_end = conf.GetAtomPosition(end).x, conf.GetAtomPosition(end).y

        x_start, y_start = int((x_start - x_trans) * scale + dim/2), int((y_start - y_trans) * scale + dim/2)
        x_end, y_end = int((x_end - x_trans) * scale + dim/2), int((y_end - y_trans) * scale + dim/2)

        # Drawing a simple line between start and end (there are more efficient ways for anti-aliased lines, etc.)
        dx, dy = x_end - x_start, y_end - y_start
        dist = int(np.sqrt(dx*dx + dy*dy))
        for i in range(dist):
            x, y = x_start + i*dx/dist, y_start + i*dy/dist
            if 0 <= x < dim and 0 <= y < dim:
                image[int(y), int(x), 1] = 2  # Bond channel (channel 1)

    return torch.tensor(image).permute(2, 0, 1)  # Channels first format

class PredictMoleculeDataset(Dataset):
    def __init__(self, tensors, smiles):
        self.tensors = tensors
        self.smiles = smiles

    def __len__(self):
        return len(self.tensors)

    def __getitem__(self, idx):
        return self.tensors[idx], self.smiles[idx]

def prepare_dataset(smiles_list):
    data_tensors = []
    successful_smiles = []

    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            image_tensor = generate_chemception_image(mol)
            if image_tensor is not None:  # Ensure embedding was successful
                data_tensors.append(image_tensor)
                successful_smiles.append(smiles)

    data_tensor = torch.stack(data_tensors).float()

    # Create the dataset
    dataset = PredictMoleculeDataset(data_tensor, successful_smiles)

    return dataset

# Categorise only values classified with probability that is over a certain treshold
def categorize_output(raw_output):
  prob = torch.sigmoid(raw_output).item()
  if prob > 0.8:
      return 1
  elif prob < 0.2:
      return 0
  else:
      return 'unsure'

def predict_all_molecules(model, data_loader):
    active_outputs = []
    inactive_outputs = []
    all_molecule_smiles = []

    with torch.no_grad():
        for inputs, smiles_batch in data_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)

            for idx, raw_output in enumerate(outputs):
                category = categorize_output(raw_output)
                if category == 1:
                    active_outputs.append((torch.sigmoid(raw_output).item(), smiles_batch[idx]))
                elif category == 0:
                    inactive_outputs.append((torch.sigmoid(raw_output).item(), smiles_batch[idx]))

    # Sort predictions and molecule_smiles together
    sorted_active_molecules = sorted(active_outputs, key=lambda x: x[0], reverse=True)
    sorted_inactive_molecules = sorted(inactive_outputs, key=lambda x: x[0])

    # Get top 10 active and inactive molecules
    active_smiles = [item[1] for item in sorted_active_molecules]
    inactive_smiles = [item[1] for item in sorted_inactive_molecules]

    return active_smiles, inactive_smiles

# Only get molecules that have been consistently classified as active/inactive
def get_consistent_molecules(model, data_loader, runs=5):
    all_active = []
    all_inactive = []

    for _ in range(runs):
        active, inactive = predict_all_molecules(model, data_loader)
        all_active.extend(active)
        all_inactive.extend(inactive)

    active_counts = Counter(all_active)
    inactive_counts = Counter(all_inactive)

    consistent_active = [smiles for smiles, count in active_counts.items() if count >= 4]
    consistent_inactive = [smiles for smiles, count in inactive_counts.items() if count >= 4]

    return consistent_active, consistent_inactive

In [ ]:
def get_mol(smiles_or_mol):
    '''
    Loads SMILES/molecule into RDKit's object
    '''
    if isinstance(smiles_or_mol, str):
        if len(smiles_or_mol) == 0:
            return None
        mol = Chem.MolFromSmiles(smiles_or_mol)
        if mol is None:
            return None
        try:
            Chem.SanitizeMol(mol)
        except:
            return None
        return mol
    return smiles_or_mol

In [ ]:
# Get all the metrics of interest for a list of SMILES
def evaluate_smiles(smiles_list, expert_smiles, training_smiles):
  valid_smiles = [s for s in smiles_list if get_mol(s) is not None]
  prepared_dataset = prepare_dataset(set(valid_smiles))
  data_loader = DataLoader(prepared_dataset, batch_size=128, shuffle=False)
  consistent_actives, consistent_inactives = get_consistent_molecules(model, data_loader)
  #print("Consistently Top Active SMILES:", consistent_actives)
  print("Nr of consistently active SMILES: ", len(consistent_actives))
  #print("Consistently Top Inactive SMILES:", consistent_inactives)
  print("Nr of consistently inactive SMILES: ", len(consistent_inactives))
  fpgen = AllChem.GetRDKitFPGenerator()
  generated_mols = [get_mol(smiles) for smiles in smiles_list]
  expert_mols = [get_mol(smiles) for smiles in expert_smiles]
  validity = sum([1 if mol is not None else 0 for mol in generated_mols]) / len(generated_mols)
  generated_mols = [mol for mol in generated_mols if mol is not None]
  expert_mols = [mol for mol in expert_mols if mol is not None]
  print("Validity: ", validity)
  unique_mols = len(set(smiles_list)) / len(smiles_list)
  print("Uniqueness: ", unique_mols)
  novel_mols = len(set(smiles_list) - set(expert_smiles) - set(training_smiles)) / len(smiles_list)
  print("Novelty: ", novel_mols)
  max_tanimoto_sims = []
  for gm in generated_mols:
      gm_fp = AllChem.GetMorganFingerprintAsBitVect(gm, radius=2)  # Using Morgan fingerprints
      sim_scores = [TanimotoSimilarity(gm_fp, AllChem.GetMorganFingerprintAsBitVect(mol, radius=2)) for mol in expert_mols]
      max_tanimoto_sims.append(max(sim_scores))
  average_max_tanimoto_similarity = sum(max_tanimoto_sims) / len(max_tanimoto_sims)
  print("Average max Tanimoto similarity to experts: ", average_max_tanimoto_similarity)
  print("Best Tanimoto similarity to experts: ", max(max_tanimoto_sims))
  sas_scores = [sascorer.calculateScore(mol) for mol in generated_mols if mol is not None]
  average_sas_score = sum(sas_scores) / len(sas_scores)
  print("Average synthesisability: ", average_sas_score)
  qed_scores = [QED.qed(mol) for mol in generated_mols if mol is not None]
  average_qed_score = sum(qed_scores) / len(qed_scores)
  print("Average QED: ", average_qed_score)
  return len(consistent_actives), len(consistent_inactives), validity, unique_mols, novel_mols, average_max_tanimoto_similarity, max(max_tanimoto_sims), average_sas_score, average_qed_score

In [ ]:
# Evaluate for a specific list of sampled SMILES
def set_and_evaluate(model_name, agent_type):
  vae_model = model_name
  agent = agent_type
  df = pd.read_csv('./data/sampled_data/'+vae_model+'/'+agent+'/40000_all.csv')
  smiles_list = df['0']
  experts = pd.read_csv('./data/canonized/esr1.csv')
  expert_smiles = experts['SMILES'].tolist()
  if "biased" in vae_model:
    training_df = pd.read_csv('./data/canonized/moses_biased_train_50k.csv')
  else:
    training_df = pd.read_csv('./data/canonized/moses_train_50k.csv')
  training_smiles = training_df['SMILES'].tolist()
  return evaluate_smiles(smiles_list, expert_smiles, training_smiles)

In [ ]:
column_names = ["Nr Active", "Nr Inactive", "Validity", "Uniqueness",
                "Novelty", "Avg Max Tanimoto Sim", "Best Tanimoto Sim",
                "Avg SA", "Avg QED"]
results_df = df = pd.DataFrame(columns=column_names)

In [ ]:
data_dict = {col: val for col, val in zip(results_df.columns, set_and_evaluate("009_random_cvae_rnn_50k", "IL4"))}
results_df = results_df.append(pd.DataFrame([data_dict], index=["Random CVAE"]))

data_dict = {col: val for col, val in zip(results_df.columns, set_and_evaluate("010_biased_cvae_rnn_50k", "IL4"))}
results_df = results_df.append(pd.DataFrame([data_dict], index=["Biased CVAE"]))

data_dict = {col: val for col, val in zip(results_df.columns, set_and_evaluate("011_random_vae_rnn_50k", "IL4"))}
results_df = results_df.append(pd.DataFrame([data_dict], index=["Random VAE"]))

data_dict = {col: val for col, val in zip(results_df.columns, set_and_evaluate("012_biased_vae_rnn_50k", "IL4"))}
results_df = results_df.append(pd.DataFrame([data_dict], index=["Biased VAE"]))

Embedding failed for molecule with SMILES: [H]c1c([H])c([H])c(C([H])([H])C2([H])C(=O)Oc3c([H])c([H])c(N([H])[H])c2c3[H])c([H])c1[H]
Nr of consistently active SMILES:  163
Nr of consistently inactive SMILES:  367
Validity:  0.896
Uniqueness:  0.947
Novelty:  0.946
Average max Tanimoto similarity to experts:  0.24610619399918301
Best Tanimoto similarity to experts:  0.5116279069767442
Average synthesisability:  2.0433204159557885
Average QED:  0.7869646616578859


<ipython-input-23-7afa43112d70>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(pd.DataFrame([data_dict], index=["Random CVAE"]))


Nr of consistently active SMILES:  166
Nr of consistently inactive SMILES:  350
Validity:  0.858
Uniqueness:  0.92
Novelty:  0.917
Average max Tanimoto similarity to experts:  0.22633502128829758
Best Tanimoto similarity to experts:  0.3815789473684211
Average synthesisability:  2.448251629455924
Average QED:  0.8463012953378274


<ipython-input-23-7afa43112d70>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(pd.DataFrame([data_dict], index=["Biased CVAE"]))


Nr of consistently active SMILES:  214
Nr of consistently inactive SMILES:  300
Validity:  0.888
Uniqueness:  0.884
Novelty:  0.882
Average max Tanimoto similarity to experts:  0.2683947935842502
Best Tanimoto similarity to experts:  0.6153846153846154
Average synthesisability:  2.0184120249379682
Average QED:  0.8710805765841967


<ipython-input-23-7afa43112d70>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(pd.DataFrame([data_dict], index=["Random VAE"]))


Embedding failed for molecule with SMILES: [H]c1c([H])c([H])c(C([H])([H])C([H])([H])[H])c(N2c3c([H])c([H])c(c([H])c3C([H])([H])OC([H])([H])[H])S2(=O)=O)c1[H]
Nr of consistently active SMILES:  253
Nr of consistently inactive SMILES:  309
Validity:  0.899
Uniqueness:  0.968
Novelty:  0.967
Average max Tanimoto similarity to experts:  0.2332840770615534
Best Tanimoto similarity to experts:  0.43548387096774194
Average synthesisability:  2.1248163663330843
Average QED:  0.8291545493273829


<ipython-input-23-7afa43112d70>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(pd.DataFrame([data_dict], index=["Biased VAE"]))


In [ ]:
results_df

,Nr Active,Nr Inactive,Validity,Uniqueness,Novelty,Avg Max Tanimoto Sim,Best Tanimoto Sim,Avg SA,Avg QED
Random CVAE,163,367,0.896,0.947,0.946,0.246106,0.511628,2.043320,0.786965
Biased CVAE,166,350,0.858,0.920,0.917,0.226335,0.381579,2.448252,0.846301
Random VAE,214,300,0.888,0.884,0.882,0.268395,0.615385,2.018412,0.871081
Biased VAE,253,309,0.899,0.968,0.967,0.233284,0.435484,2.124816,0.829155


In [ ]:
results_df.to_csv('./models/metrics/IL4/eval_metrics.csv')